## Final Project Submission

Please fill out: 
* Student name: **Alberto SYLVEUS** 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: **Wedter JEROME**
*  
[Click here to get the github repository](https://github.com/tolly00/dsc-phase2-myproject.git)


## Overview

In this project, we will use exploratory data analysis to generate insight for a business stackholder. 

## Business Understanding

After seing all the big company creating original video content and the want ti get in on the fun. A company has decided to create a new movie studio, but the personal doesn't know anything about creating movies. So we are going to exploring what types of films are currently doing the best at the box office. we will translate those findings into actionable insights that the head of the company's new movie studio can use to help decide what type of films to create.

## Data Understanding

* We have data from various locations, the different files have different formats. Some are compressed CSV(comma-separated values) or TSV(tab-separated values) files that can be opened using spreadsheet software or `pd.read_csv`, whith the data from IMDB is located in a SQLite database. In this case all those data are zippep in a folder that called `zippedData`, we will use `file explorer` to unzip them. Right now those files are ready to use.

    The goal of this analysis is to identify what types of films are currently doing the best at the box office to help the company's new movie studio about their decisions to create movie.

* In `zippedData` we are going to use `im.db` file. This is a sqlite database file. This database has 8 tables: movies_basics, directors, known_for, movies_akas, movies_rating, persons, principals, writers.
* The tables that have relevant data to make this analysis possible is: **movie_basics** and **movie_rating**.

  - **movie_basics:** has data about title, runtime, genres, etc...
  - **movie_rating**: has data about average rating and number of votes
* film with more votes

* film with more rating

* more common type film

* runtimes minutes with numvotes and average raating

## Data Preparation

In [1]:
# import necessary modules
import pandas as pd
import sqlite3

In [4]:
# connect to the database
path = "zippedData/im.db"
conn = sqlite3.connect(path)
q = """
    SELECT name
    FROM sqlite_master
    WHERE type = "table"
    """
pd.read_sql(q,conn) #show all tables name of the database

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


Let's use `movie_basics` and `movie_rating` tables to create a view.To avoid compromizing our analysis, we drop rows with NaN values.

In [7]:
# join them together
q = """
    CREATE VIEW IF NOT EXISTS movies AS
    SELECT
    b.movie_id, b.primary_title, b.start_year, b.genres, b.runtime_minutes, r.averagerating, r.numvotes 
    FROM movie_basics AS b
    JOIN movie_ratings AS r
    USING (movie_id)
    WHERE 
    b.genres IS NOT NULL AND b.genres != ''
    AND b.runtime_minutes IS NOT NULL
    AND r.averagerating IS NOT NULL
    AND r.numvotes IS NOT NULL;
    """
conn.execute(q)
conn.commit()

In [5]:
# show the movies view
q = """
SELECT * FROM movies;
"""
df_movies = pd.read_sql(q,conn)
df_movies.head()

,movie_id,primary_title,start_year,genres,runtime_minutes,averagerating,numvotes
0,tt0063540,Sunghursh,2013,"Action,Crime,Drama",175.0,7.0,77
1,tt0066787,One Day Before the Rainy Season,2019,"Biography,Drama",114.0,7.2,43
2,tt0069049,The Other Side of the Wind,2018,Drama,122.0,6.9,4517
3,tt0100275,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",80.0,6.5,119
4,tt0137204,Joe Finds Grace,2017,"Adventure,Animation,Comedy",83.0,8.1,263


In [6]:
# A quickly preview on data after cleaning
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65720 entries, 0 to 65719
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         65720 non-null  object 
 1   primary_title    65720 non-null  object 
 2   start_year       65720 non-null  int64  
 3   genres           65720 non-null  object 
 4   runtime_minutes  65720 non-null  float64
 5   averagerating    65720 non-null  float64
 6   numvotes         65720 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 3.5+ MB


## Analysis and results

We are going to work on the dataset to see what types of films are doing the best at the box office.

In [7]:
df_movies.head()

,movie_id,primary_title,start_year,genres,runtime_minutes,averagerating,numvotes
0,tt0063540,Sunghursh,2013,"Action,Crime,Drama",175.0,7.0,77
1,tt0066787,One Day Before the Rainy Season,2019,"Biography,Drama",114.0,7.2,43
2,tt0069049,The Other Side of the Wind,2018,Drama,122.0,6.9,4517
3,tt0100275,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",80.0,6.5,119
4,tt0137204,Joe Finds Grace,2017,"Adventure,Animation,Comedy",83.0,8.1,263


In the `genres` column we can see certains of films have multiples genres. we are going to make a split on it to help us grouping by `genres`.

In [10]:
df_movies['genres'] = df_movies['genres'].str.split(',') # split comma-separated genres into lists

In [12]:
df_exploded = df_movies.explode('genres') # explode the list into separate rows

In [16]:
df_exploded.info() # a quickly preview of the dataset update.

<class 'pandas.core.frame.DataFrame'>
Index: 118437 entries, 0 to 65719
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         118437 non-null  object 
 1   primary_title    118437 non-null  object 
 2   start_year       118437 non-null  int64  
 3   genres           118437 non-null  object 
 4   runtime_minutes  118437 non-null  float64
 5   averagerating    118437 non-null  float64
 6   numvotes         118437 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 7.2+ MB


In [24]:
# group data by genres sort by numvotes
top_genre_vote = df_exploded.groupby('genres').agg({
    'averagerating': 'mean',
    'numvotes': 'sum'
}).sort_values(by=['numvotes','averagerating'], ascending=False)
top_genre_vote.head()

,averagerating,numvotes
genres,,
Drama,6.391692,119452909
Action,5.784437,101126583
Adventure,6.178873,84222953
Comedy,6.014110,74181319
Thriller,5.592140,48126681


Here we sort genres by `numvotes`. numvotes is the column that show us the numbers of people that whatch the movie. Where we consider this option the top 5 of the best types of movies at the box offices are: **Drama, Action, Adventure, Comedy Thriller.**

In [23]:
# group data by genres sort by average rating
top_genre_avgrating = df_exploded.groupby('genres').agg({
    'averagerating': 'mean',
    'numvotes': 'sum'
}).sort_values(by='averagerating', ascending=False)
top_genre_avgrating.head( 

,averagerating,numvotes
genres,,
Short,8.800000,8
Documentary,7.316787,4709902
Game-Show,7.300000,3469
News,7.278783,123284
Biography,7.169185,21603178


When we consider sorting by `averagerating`. average rating help us to know how the movie is appreciated. so a movie can have a lot of watching and people don't like it. if we want to know about the appreciation we have to look at the average rating. According that the top 5 of the best types of films are: **Short, Documentary, Game-Show, News, Biography.**

Now, let's take a look at the top 10 `films` by popularity votes.

Let's see about the top 10 by `averagerating`.

In [17]:
top_avgrating = """
SELECT primary_title, averagerating
FROM movies
ORDER BY averagerating DESC
LIMIT 20;
"""
df_top_avgrating = pd.read_sql(top_avgrating,conn)
df_top_avgrating

,primary_title,averagerating
0,Exteriores: Mulheres Brasileiras na Diplomacia,10.0
1,The Dark Knight: The Ballad of the N Word,10.0
2,Freeing Bernie Baran,10.0
3,Hercule contre Hermès,10.0
4,I Was Born Yesterday!,10.0
5,Revolution Food,10.0
6,Fly High: Story of the Disc Dog,10.0
7,The Paternal Bond: Barbary Macaques,10.0
8,Requiem voor een Boom,10.0
9,A Dedicated Life: Phoebe Brand Beyond the Group,10.0


### Business Recommendation 1

### Business Recommendation 2

### Business Recommendation 3

### Vizualisation

## Conclusion

## Next step